# Yuhua Sandbox

## Algorithm to Automate Novelty Scoring

### Import Packages

In [1]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from functools import reduce
import openpyxl
import xlsxwriter

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.util import ngrams
from nltk import ngrams, FreqDist
from nltk.lm import NgramCounter
import string
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

from spacy.lang.en.stop_words import STOP_WORDS

from collections import Counter
import itertools

from sklearn.metrics.pairwise import cosine_similarity

from scipy import spatial

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt
from yellowbrick.cluster import KElbowVisualizer

from nltk.cluster.kmeans import KMeansClusterer

from scipy import stats
from scipy.stats import pearsonr

/Users/yyu/opt/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import os

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/yyu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/yyu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/yyu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### Put Data from Excel Sheet into Dataframes

In [4]:
# individual df's for each sheet

# when on pc
data_cup = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test_cup_semdis.csv")
data_key = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test_key_semdis.csv")
data_rope = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test_rope_semdis.csv")
data_brick = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test_brick_semdis.csv")
data_chair = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test_chair_semdis.csv")
data_pencil = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test_pencil_semdis.csv")
data_shoe = pd.read_csv("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test_shoe_semdis.csv")

# when on mac
# data_cup = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test_cup_semdis.csv.xlsx")
# data_key = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test_key_semdis.csv.xlsx")
# data_rope = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test_rope_semdis.csv.xlsx")
# data_brick = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test_brick_semdis.csv.xlsx")
# data_chair = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test_chair_semdis.csv.xlsx")
# data_pencil = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test_pencil_semdis.csv.xlsx")
# data_shoe = pd.read_csv("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test_shoe_semdis.csv.xlsx")

In [7]:
cwd = os.getcwd()
data_cup = pd.read_csv(cwd+'/../data/autdata_test_cup_semdis.csv')

data_key = pd.read_csv(cwd+'/../data/autdata_test_key_semdis.csv')
data_rope = pd.read_csv(cwd+'/../data/autdata_test_rope_semdis.csv')
data_brick = pd.read_csv(cwd+'/../data/autdata_test_brick_semdis.csv')
data_chair = pd.read_csv(cwd+'/../data/autdata_test_chair_semdis.csv')
data_pencil = pd.read_csv(cwd+'/../data/autdata_test_pencil_semdis.csv')
data_shoe = pd.read_csv(cwd+'/../data/autdata_test_shoe_semdis.csv')

### Diversity Calc

In [ ]:

#compute centroid of the list of vectors, as the average of the normalized vectors
def calc_centroid(vector_list):
    vector_len=300 #the length of the embedded vector
    centroid=np.zeros(vector_len)
    for v in vector_list:
        centroid=centroid+v/np.linalg.norm(v)
    centroid=centroid/len(vector_list)
    return centroid

#main function to compute the diversity metric, input is the responses from same participant for a single prompt
#each response is already converted to embedded vectors using certain composition
def calc_diversity(vector_list):
    centroid=calc_centroid(vector_list)
    dist=[]
    for v in vector_list:
        dist.append(1-get_cosine_similarity(centroid,v))
    #return the max of all distance, root mean square of the distance
    #they are equivalent in some sense, root mean squre might behave sligthly better as the diversity metric
    return np.max(dist),np.sqrt(np.mean(np.array(dist)**2))